In [ ]:
# =============================================================================
# =============================================================================
#
#  CAPITAL ALLOCATION EFFICIENCY SCORECARD
#  Magnificent 7 AI-Related CapEx Analysis (FY2022-2024)
#
#  Author: Ekkarit Gaewprapun, DBA, CMA, FMVA
#  Version: 1.0
#
# =============================================================================
# =============================================================================

# %% [markdown]
"""
# Capital Allocation Efficiency Scorecard
## Magnificent 7 AI-Related CapEx Analysis (FY2022-2024)

**Author**: Ekkarit Gaewprapun, DBA, CMA, FMVA
**Last Updated**: [Date]
**Version**: 1.0

---

### Executive Summary

This notebook implements a four-dimension scoring framework to evaluate
how efficiently the Magnificent Seven technology companies convert
AI-related capital expenditure into sustainable financial returns.

### Key Findings (FY2024)

[Auto-generated summary will appear here after model runs]

### Methodology Overview

The scorecard evaluates companies across four dimensions:
1. **Capital Deployment Scale** — Magnitude and trajectory of investment
2. **Conversion Efficiency** — Speed of capital-to-revenue translation
3. **Return Quality** — Margin preservation during scaling
4. **Sustainability & Risk** — Capacity to maintain investment intensity

### Data Sources

- SEC 10-K filings (FY2022, FY2023, FY2024)
- Earnings call transcripts (FY2025 guidance)
- Company segment disclosures

---
"""

# ============================================================================
# DATA VALIDATION SCRIPT
# Capital Allocation Efficiency Scorecard
# Run this BEFORE executing the main notebook
# ============================================================================

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import os

# Configuration
DATA_PATH = '/content/drive/MyDrive/Projects/CapEx_Scorecard/data/'

# Expected schemas
EXPECTED_FILES = {
    'consolidated_financials.csv': {
        'expected_rows': 21,
        'required_columns': ['ticker', 'fiscal_year', 'revenue', 'cost_of_revenue',
                            'gross_profit', 'rd_expense', 'operating_income', 'net_income',
                            'total_assets', 'total_debt', 'cash_and_equivalents',
                            'operating_cash_flow', 'capex', 'depreciation_amortization']
    },
    'segment_financials.csv': {
        'expected_rows': 48,
        'required_columns': ['ticker', 'fiscal_year', 'segment_name',
                            'segment_revenue', 'segment_operating_income']
    },
    'forward_guidance.csv': {
        'expected_rows': 7,
        'required_columns': ['ticker', 'guidance_year', 'capex_guidance_low',
                            'capex_guidance_high', 'capex_guidance_midpoint',
                            'source', 'source_date']
    },
    'purchase_obligations.csv': {
        'expected_rows': 21,
        'required_columns': ['ticker', 'fiscal_year', 'purchase_obligations_total',
                            'purchase_obligations_1yr', 'operating_lease_obligations',
                            'finance_lease_obligations']
    }
}

EXPECTED_COMPANIES = ['AAPL', 'AMZN', 'GOOGL', 'META', 'MSFT', 'NVDA', 'TSLA']
EXPECTED_YEARS = [2022, 2023, 2024]

# ============================================================================
# VALIDATION FUNCTIONS
# ============================================================================

def validate_file_exists(filename):
    """Check if file exists and return DataFrame if so."""
    filepath = DATA_PATH + filename
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        return True, df
    return False, None

def validate_columns(df, required_columns, filename):
    """Check for missing columns."""
    missing = [col for col in required_columns if col not in df.columns]
    extra = [col for col in df.columns if col not in required_columns]
    return missing, extra

def validate_row_count(df, expected_rows):
    """Check row count."""
    return len(df), expected_rows, len(df) == expected_rows

def validate_companies(df):
    """Check all expected companies are present."""
    present = df['ticker'].unique().tolist()
    missing = [c for c in EXPECTED_COMPANIES if c not in present]
    extra = [c for c in present if c not in EXPECTED_COMPANIES]
    return missing, extra

def validate_years(df):
    """Check all expected years are present."""
    present = df['fiscal_year'].unique().tolist()
    missing = [y for y in EXPECTED_YEARS if y not in present]
    return missing

# ============================================================================
# RUN VALIDATION
# ============================================================================

print("=" * 70)
print("DATA VALIDATION REPORT")
print("Capital Allocation Efficiency Scorecard")
print("=" * 70)

errors = []
warnings = []
dataframes = {}

# Step 1: Check all files exist
print("\n1. FILE EXISTENCE CHECK")
print("-" * 40)

for filename in EXPECTED_FILES.keys():
    exists, df = validate_file_exists(filename)
    if exists:
        print(f"✓ {filename}")
        dataframes[filename] = df
    else:
        print(f"✗ MISSING: {filename}")
        errors.append(f"File not found: {filename}")

# Step 2: Validate columns
print("\n2. COLUMN VALIDATION")
print("-" * 40)

for filename, schema in EXPECTED_FILES.items():
    if filename in dataframes:
        df = dataframes[filename]
        missing, extra = validate_columns(df, schema['required_columns'], filename)
        if missing:
            print(f"✗ {filename}: Missing columns: {missing}")
            errors.append(f"{filename} missing columns: {missing}")
        elif extra:
            print(f"⚠ {filename}: Extra columns (ignored): {extra}")
            warnings.append(f"{filename} has extra columns: {extra}")
        else:
            print(f"✓ {filename}: All columns present")

# Step 3: Validate row counts
print("\n3. ROW COUNT VALIDATION")
print("-" * 40)

for filename, schema in EXPECTED_FILES.items():
    if filename in dataframes:
        df = dataframes[filename]
        actual, expected, match = validate_row_count(df, schema['expected_rows'])
        if match:
            print(f"✓ {filename}: {actual} rows (expected {expected})")
        else:
            print(f"⚠ {filename}: {actual} rows (expected {expected})")
            if actual < expected:
                warnings.append(f"{filename}: Only {actual} rows, expected {expected}")

# Step 4: Validate companies
print("\n4. COMPANY COVERAGE")
print("-" * 40)

for filename in ['consolidated_financials.csv', 'segment_financials.csv', 'purchase_obligations.csv']:
    if filename in dataframes:
        df = dataframes[filename]
        missing, extra = validate_companies(df)
        if missing:
            print(f"✗ {filename}: Missing companies: {missing}")
            errors.append(f"{filename} missing companies: {missing}")
        else:
            print(f"✓ {filename}: All 7 companies present")

# Step 5: Validate years
print("\n5. FISCAL YEAR COVERAGE")
print("-" * 40)

for filename in ['consolidated_financials.csv', 'segment_financials.csv', 'purchase_obligations.csv']:
    if filename in dataframes:
        df = dataframes[filename]
        missing = validate_years(df)
        if missing:
            print(f"✗ {filename}: Missing years: {missing}")
            errors.append(f"{filename} missing years: {missing}")
        else:
            print(f"✓ {filename}: All 3 years present (2022-2024)")

# Step 6: Cross-reference segment revenue to consolidated
print("\n6. SEGMENT-TO-CONSOLIDATED RECONCILIATION")
print("-" * 40)

if 'consolidated_financials.csv' in dataframes and 'segment_financials.csv' in dataframes:
    consol = dataframes['consolidated_financials.csv']
    segments = dataframes['segment_financials.csv']

    # Sum segment revenue by company-year
    seg_sum = segments.groupby(['ticker', 'fiscal_year'])['segment_revenue'].sum().reset_index()
    seg_sum.columns = ['ticker', 'fiscal_year', 'segment_total']

    # Merge with consolidated
    merged = consol[['ticker', 'fiscal_year', 'revenue']].merge(seg_sum, on=['ticker', 'fiscal_year'], how='left')
    merged['diff_pct'] = abs(merged['segment_total'] - merged['revenue']) / merged['revenue'] * 100

    print(f"{'Ticker':<8} {'Year':<6} {'Consol Rev':>14} {'Seg Sum':>14} {'Diff %':>8} {'Status':<10}")
    print("-" * 60)

    for _, row in merged[merged['fiscal_year'] == 2024].iterrows():
        diff_pct = row['diff_pct']
        status = "✓ OK" if diff_pct < 5 else "⚠ CHECK"
        print(f"{row['ticker']:<8} {int(row['fiscal_year']):<6} {row['revenue']:>14,.0f} {row['segment_total']:>14,.0f} {diff_pct:>7.1f}% {status:<10}")

        if diff_pct >= 5:
            warnings.append(f"{row['ticker']} FY{int(row['fiscal_year'])}: Segment sum differs by {diff_pct:.1f}%")

# Step 7: Check for NaN values in critical fields
print("\n7. DATA QUALITY CHECK (NaN VALUES)")
print("-" * 40)

critical_fields = {
    'consolidated_financials.csv': ['revenue', 'operating_income', 'capex'],
    'segment_financials.csv': ['segment_revenue'],
    'forward_guidance.csv': ['capex_guidance_midpoint']
}

for filename, fields in critical_fields.items():
    if filename in dataframes:
        df = dataframes[filename]
        for field in fields:
            if field in df.columns:
                nan_count = df[field].isna().sum()
                if nan_count > 0:
                    print(f"⚠ {filename}.{field}: {nan_count} NaN values")
                    warnings.append(f"{filename}.{field} has {nan_count} NaN values")
                else:
                    print(f"✓ {filename}.{field}: No NaN values")

# Step 8: Preview consolidated data (FY2024)
print("\n8. DATA PREVIEW: CONSOLIDATED FY2024")
print("-" * 40)

if 'consolidated_financials.csv' in dataframes:
    df = dataframes['consolidated_financials.csv']
    fy24 = df[df['fiscal_year'] == 2024][['ticker', 'revenue', 'capex', 'operating_income']].sort_values('revenue', ascending=False)
    print(fy24.to_string(index=False))

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "=" * 70)
print("VALIDATION SUMMARY")
print("=" * 70)

print(f"\nErrors: {len(errors)}")
for e in errors:
    print(f"  ✗ {e}")

print(f"\nWarnings: {len(warnings)}")
for w in warnings:
    print(f"  ⚠ {w}")

if len(errors) == 0:
    print("\n" + "=" * 70)
    print("✓ DATA READY FOR MODEL EXECUTION")
    print("  Set USE_SYNTHETIC_DATA = False and run all cells")
    print("=" * 70)
else:
    print("\n" + "=" * 70)
    print("✗ FIX ERRORS BEFORE PROCEEDING")
    print("  Review error messages above and correct data files")
    print("=" * 70)

# =============================================================================
# SECTION 1: ENVIRONMENT SETUP
# =============================================================================

# %% [markdown]
"""
## Section 1: Environment Setup

Install dependencies and configure the runtime environment.
"""

# %%
# 1.1 Install Required Packages
# -----------------------------------------------------------------------------
!pip install pandas numpy matplotlib seaborn plotly openpyxl kaleido -q
!pip install pandas numpy matplotlib seaborn plotly openpyxl python-docx -q
!pip install -U kaleido -q

# %%
# 1.2 Import Libraries
# -----------------------------------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime
from typing import Dict, List, Tuple, Optional
import os

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✓ Libraries imported successfully")

# %%
# 1.3 Google Drive Mount (for data persistence)
# -----------------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

# Define project paths
PROJECT_PATH = '/content/drive/MyDrive/Projects/CapEx_Scorecard/'
DATA_PATH = PROJECT_PATH + 'data/'
OUTPUT_PATH = PROJECT_PATH + 'outputs/'

# Create directories if they don't exist
os.makedirs(DATA_PATH, exist_ok=True)
os.makedirs(OUTPUT_PATH, exist_ok=True)

print("✓ Environment setup complete")
print(f"✓ Project path: {PROJECT_PATH}")
print(f"✓ Data path: {DATA_PATH}")
print(f"✓ Output path: {OUTPUT_PATH}")

# =============================================================================
# SECTION 2: CONFIGURATION & PARAMETERS
# =============================================================================

# %% [markdown]
"""
## Section 2: Configuration & Parameters

Centralize all adjustable parameters for easy model refresh.
"""

# %%
# 2.1 Analysis Period
# -----------------------------------------------------------------------------
FISCAL_YEARS = [2022, 2023, 2024]
CURRENT_YEAR = 2024
FORWARD_YEAR = 2025

# 2.2 Company Universe
# -----------------------------------------------------------------------------
COMPANIES = {
    'AAPL': {'name': 'Apple', 'fy_end': 'September', 'color': '#555555'},
    'MSFT': {'name': 'Microsoft', 'fy_end': 'June', 'color': '#00A4EF'},
    'GOOGL': {'name': 'Alphabet', 'fy_end': 'December', 'color': '#4285F4'},
    'AMZN': {'name': 'Amazon', 'fy_end': 'December', 'color': '#FF9900'},
    'META': {'name': 'Meta', 'fy_end': 'December', 'color': '#0668E1'},
    'NVDA': {'name': 'NVIDIA', 'fy_end': 'January', 'color': '#76B900'},
    'TSLA': {'name': 'Tesla', 'fy_end': 'December', 'color': '#CC0000'}
}

COMPANY_ORDER = ['NVDA', 'AMZN', 'MSFT', 'AAPL', 'GOOGL', 'META', 'TSLA']

# 2.3 Segment Mapping
# -----------------------------------------------------------------------------
# Defines which segments to analyze for each company
SEGMENT_CONFIG = {
    'MSFT': {
        'segments': ['Intelligent Cloud', 'Productivity and Business Processes',
                     'More Personal Computing'],
        'ai_primary': 'Intelligent Cloud'
    },
    'GOOGL': {
        'segments': ['Google Services', 'Google Cloud', 'Other Bets'],
        'ai_primary': 'Google Cloud'
    },
    'AMZN': {
        'segments': ['AWS', 'North America', 'International'],
        'ai_primary': 'AWS'
    },
    'META': {
        'segments': ['Family of Apps', 'Reality Labs'],
        'ai_primary': 'Reality Labs'
    },
    'NVDA': {
        'segments': ['Compute & Networking', 'Graphics'],
        'ai_primary': 'Compute & Networking'
    },
    'AAPL': {
        'segments': ['Consolidated'],  # No segment-level P&L
        'ai_primary': 'Consolidated'
    },
    'TSLA': {
        'segments': ['Automotive', 'Energy Generation and Storage'],
        'ai_primary': 'Automotive'
    }
}

# 2.4 Scoring Weights
# -----------------------------------------------------------------------------
DIMENSION_WEIGHTS = {
    'capital_deployment': 0.15,
    'conversion_efficiency': 0.35,
    'return_quality': 0.30,
    'sustainability_risk': 0.20
}

# Verify weights sum to 1.0
assert abs(sum(DIMENSION_WEIGHTS.values()) - 1.0) < 0.001, "Dimension weights must sum to 1.0"

# 2.5 Classification Thresholds
# -----------------------------------------------------------------------------
CLASSIFICATION_THRESHOLDS = {
    'Capital Efficient': (7.0, 10.0),
    'Capital Intensive': (4.0, 6.99),
    'Capital Dependent': (1.0, 3.99)
}

# 2.6 Display Settings
# -----------------------------------------------------------------------------
CHART_TEMPLATE = 'plotly_white'
FIGURE_DPI = 150

print("✓ Configuration loaded")
print(f"✓ Analyzing {len(COMPANIES)} companies across {len(FISCAL_YEARS)} fiscal years")
print(f"✓ Dimension weights: {DIMENSION_WEIGHTS}")

# =============================================================================
# SECTION 2.5: SYNTHETIC DATA GENERATOR (Development Mode)
# =============================================================================

# %% [markdown]
"""
## Section 2.5: Synthetic Data Generator (Development Mode)

**PURPOSE**: Generate realistic synthetic data for development and testing.

**USAGE**:
- Set `USE_SYNTHETIC_DATA = True` to generate and use synthetic data
- Set `USE_SYNTHETIC_DATA = False` once real data is available

**NOTE**: Replace with real SEC filing data before final analysis.
"""

# %%
# Toggle for synthetic vs. real data
USE_SYNTHETIC_DATA = False  # Set to False when real data is ready

# %%
# 2.5.1 Realistic Base Values (FY2024 Approximate Scale)
# -----------------------------------------------------------------------------
# Based on publicly known ranges to make synthetic data realistic
# All values in millions USD

BASE_FINANCIALS = {
    'AAPL': {
        'revenue': 385000, 'gross_margin': 0.46, 'op_margin': 0.30,
        'capex_intensity': 0.03, 'rd_intensity': 0.07, 'cash': 65000,
        'debt': 110000, 'assets': 350000
    },
    'MSFT': {
        'revenue': 245000, 'gross_margin': 0.70, 'op_margin': 0.44,
        'capex_intensity': 0.15, 'rd_intensity': 0.12, 'cash': 80000,
        'debt': 50000, 'assets': 470000
    },
    'GOOGL': {
        'revenue': 340000, 'gross_margin': 0.57, 'op_margin': 0.30,
        'capex_intensity': 0.12, 'rd_intensity': 0.14, 'cash': 110000,
        'debt': 30000, 'assets': 430000
    },
    'AMZN': {
        'revenue': 620000, 'gross_margin': 0.47, 'op_margin': 0.09,
        'capex_intensity': 0.10, 'rd_intensity': 0.14, 'cash': 90000,
        'debt': 135000, 'assets': 550000
    },
    'META': {
        'revenue': 160000, 'gross_margin': 0.81, 'op_margin': 0.35,
        'capex_intensity': 0.20, 'rd_intensity': 0.27, 'cash': 65000,
        'debt': 35000, 'assets': 230000
    },
    'NVDA': {
        'revenue': 130000, 'gross_margin': 0.73, 'op_margin': 0.62,
        'capex_intensity': 0.04, 'rd_intensity': 0.15, 'cash': 50000,
        'debt': 10000, 'assets': 100000
    },
    'TSLA': {
        'revenue': 97000, 'gross_margin': 0.18, 'op_margin': 0.08,
        'capex_intensity': 0.10, 'rd_intensity': 0.06, 'cash': 30000,
        'debt': 10000, 'assets': 110000
    }
}

# Growth assumptions for synthetic history (annualized)
GROWTH_RATES = {
    'AAPL': {'revenue': 0.02, 'capex': 0.05},
    'MSFT': {'revenue': 0.15, 'capex': 0.35},
    'GOOGL': {'revenue': 0.12, 'capex': 0.25},
    'AMZN': {'revenue': 0.12, 'capex': 0.20},
    'META': {'revenue': 0.20, 'capex': 0.40},
    'NVDA': {'revenue': 1.20, 'capex': 0.50},  # Exceptional growth
    'TSLA': {'revenue': 0.15, 'capex': 0.10}
}

# Segment allocation percentages
SEGMENT_ALLOCATION = {
    'MSFT': {
        'Intelligent Cloud': {'revenue_share': 0.43, 'op_margin': 0.45},
        'Productivity and Business Processes': {'revenue_share': 0.32, 'op_margin': 0.47},
        'More Personal Computing': {'revenue_share': 0.25, 'op_margin': 0.35}
    },
    'GOOGL': {
        'Google Services': {'revenue_share': 0.88, 'op_margin': 0.36},
        'Google Cloud': {'revenue_share': 0.11, 'op_margin': 0.05},
        'Other Bets': {'revenue_share': 0.01, 'op_margin': -3.00}
    },
    'AMZN': {
        'AWS': {'revenue_share': 0.17, 'op_margin': 0.30},
        'North America': {'revenue_share': 0.62, 'op_margin': 0.05},
        'International': {'revenue_share': 0.21, 'op_margin': -0.02}
    },
    'META': {
        'Family of Apps': {'revenue_share': 0.98, 'op_margin': 0.50},
        'Reality Labs': {'revenue_share': 0.02, 'op_margin': -7.00}
    },
    'NVDA': {
        'Compute & Networking': {'revenue_share': 0.88, 'op_margin': 0.68},
        'Graphics': {'revenue_share': 0.12, 'op_margin': 0.45}
    },
    'TSLA': {
        'Automotive': {'revenue_share': 0.85, 'op_margin': 0.10},
        'Energy Generation and Storage': {'revenue_share': 0.15, 'op_margin': 0.15}
    },
    'AAPL': {
        'Consolidated': {'revenue_share': 1.00, 'op_margin': 0.30}
    }
}

# Forward guidance estimates for FY2025 (millions USD)
FORWARD_GUIDANCE_2025 = {
    'AAPL': {'low': 12000, 'high': 14000, 'source': 'Q4 FY2024 Earnings Call'},
    'MSFT': {'low': 80000, 'high': 85000, 'source': 'Q2 FY2025 Earnings Call'},
    'GOOGL': {'low': 55000, 'high': 60000, 'source': 'Q4 2024 Earnings Call'},
    'AMZN': {'low': 100000, 'high': 110000, 'source': 'Q4 2024 Earnings Release'},
    'META': {'low': 60000, 'high': 65000, 'source': 'Q4 2024 Earnings Call'},
    'NVDA': {'low': 8000, 'high': 12000, 'source': 'Q3 FY2025 Earnings Call'},
    'TSLA': {'low': 10000, 'high': 12000, 'source': 'Q4 2024 Earnings Call'}
}

# %%
# 2.5.2 Generate Consolidated Financials
# -----------------------------------------------------------------------------
def generate_synthetic_consolidated() -> pd.DataFrame:
    """
    Generate 3 years of synthetic consolidated financial data.
    Works backward from FY2024 base values.

    Returns:
        DataFrame with consolidated financial metrics for all companies and years
    """
    np.random.seed(42)  # Reproducibility

    records = []

    for ticker, base in BASE_FINANCIALS.items():
        growth = GROWTH_RATES[ticker]

        for year_offset, year in enumerate([2022, 2023, 2024]):
            # Calculate values working backward from 2024
            years_from_base = 2024 - year

            # Revenue with growth applied backward
            revenue = base['revenue'] / ((1 + growth['revenue']) ** years_from_base)

            # Add some noise (±3%)
            revenue *= (1 + np.random.uniform(-0.03, 0.03))

            # Derived metrics
            gross_profit = revenue * base['gross_margin'] * (1 + np.random.uniform(-0.02, 0.02))
            cost_of_revenue = revenue - gross_profit

            rd_expense = revenue * base['rd_intensity'] * (1 + np.random.uniform(-0.01, 0.01))

            operating_income = revenue * base['op_margin'] * (1 + np.random.uniform(-0.03, 0.03))

            # Net income (rough approximation)
            net_income = operating_income * 0.80  # Effective tax ~20%

            # CapEx with growth
            capex = revenue * base['capex_intensity'] / ((1 + growth['capex']) ** years_from_base)
            capex *= (1 + np.random.uniform(-0.05, 0.05))
            capex = max(capex, revenue * 0.01)  # Minimum 1% of revenue

            # Cash flow (OCF typically 1.2-1.5x net income for tech)
            ocf_multiple = np.random.uniform(1.2, 1.5)
            operating_cash_flow = net_income * ocf_multiple

            # Balance sheet items (less volatile)
            assets = base['assets'] / ((1 + 0.10) ** years_from_base)
            assets *= (1 + np.random.uniform(-0.05, 0.05))

            cash = base['cash'] * (1 + np.random.uniform(-0.10, 0.10))
            debt = base['debt'] * (1 + np.random.uniform(-0.05, 0.05))

            # D&A (rough: 5-10% of assets)
            depreciation = assets * np.random.uniform(0.05, 0.10)

            records.append({
                'ticker': ticker,
                'fiscal_year': year,
                'revenue': round(revenue, 0),
                'cost_of_revenue': round(cost_of_revenue, 0),
                'gross_profit': round(gross_profit, 0),
                'rd_expense': round(rd_expense, 0),
                'operating_income': round(operating_income, 0),
                'net_income': round(net_income, 0),
                'total_assets': round(assets, 0),
                'total_debt': round(debt, 0),
                'cash_and_equivalents': round(cash, 0),
                'operating_cash_flow': round(operating_cash_flow, 0),
                'capex': round(capex, 0),
                'depreciation_amortization': round(depreciation, 0)
            })

    return pd.DataFrame(records)

# %%
# 2.5.3 Generate Segment Financials
# -----------------------------------------------------------------------------
def generate_synthetic_segments(df_consolidated: pd.DataFrame) -> pd.DataFrame:
    """
    Generate segment-level data based on consolidated totals.

    Args:
        df_consolidated: DataFrame with consolidated financial data

    Returns:
        DataFrame with segment-level revenue and operating income
    """
    np.random.seed(43)

    records = []

    for _, row in df_consolidated.iterrows():
        ticker = row['ticker']
        year = row['fiscal_year']
        total_revenue = row['revenue']

        if ticker in SEGMENT_ALLOCATION:
            for segment_name, alloc in SEGMENT_ALLOCATION[ticker].items():
                # Segment revenue
                seg_revenue = total_revenue * alloc['revenue_share']
                seg_revenue *= (1 + np.random.uniform(-0.02, 0.02))

                # Segment operating income
                seg_op_income = seg_revenue * alloc['op_margin']
                seg_op_income *= (1 + np.random.uniform(-0.05, 0.05))

                records.append({
                    'ticker': ticker,
                    'fiscal_year': year,
                    'segment_name': segment_name,
                    'segment_revenue': round(seg_revenue, 0),
                    'segment_operating_income': round(seg_op_income, 0)
                })

    return pd.DataFrame(records)

# %%
# 2.5.4 Generate Forward Guidance
# -----------------------------------------------------------------------------
def generate_synthetic_forward_guidance() -> pd.DataFrame:
    """
    Generate forward CapEx guidance data.

    Returns:
        DataFrame with FY2025 CapEx guidance for all companies
    """
    records = []

    for ticker, guidance in FORWARD_GUIDANCE_2025.items():
        records.append({
            'ticker': ticker,
            'guidance_year': 2025,
            'capex_guidance_low': guidance['low'],
            'capex_guidance_high': guidance['high'],
            'capex_guidance_midpoint': (guidance['low'] + guidance['high']) / 2,
            'source': guidance['source'],
            'source_date': '2025-01-15'  # Placeholder
        })

    return pd.DataFrame(records)

# %%
# 2.5.5 Generate Purchase Obligations
# -----------------------------------------------------------------------------
def generate_synthetic_obligations(df_consolidated: pd.DataFrame) -> pd.DataFrame:
    """
    Generate purchase obligations data.
    Typically 1-3x annual CapEx for tech companies with AI buildout.

    Args:
        df_consolidated: DataFrame with consolidated financial data

    Returns:
        DataFrame with purchase and lease obligations
    """
    np.random.seed(44)

    records = []

    # Purchase obligations multiplier varies by company strategy
    multipliers = {
        'AAPL': 1.5, 'MSFT': 2.5, 'GOOGL': 2.0, 'AMZN': 3.0,
        'META': 2.5, 'NVDA': 4.0, 'TSLA': 1.5
    }

    for _, row in df_consolidated.iterrows():
        ticker = row['ticker']
        year = row['fiscal_year']
        capex = row['capex']

        mult = multipliers.get(ticker, 2.0)

        total_obligations = capex * mult * (1 + np.random.uniform(-0.10, 0.10))
        one_year_obligations = total_obligations * 0.40  # ~40% due in 1 year

        # Lease obligations (rough approximation)
        op_lease = row['total_assets'] * np.random.uniform(0.02, 0.05)
        fin_lease = row['total_assets'] * np.random.uniform(0.01, 0.03)

        records.append({
            'ticker': ticker,
            'fiscal_year': year,
            'purchase_obligations_total': round(total_obligations, 0),
            'purchase_obligations_1yr': round(one_year_obligations, 0),
            'operating_lease_obligations': round(op_lease, 0),
            'finance_lease_obligations': round(fin_lease, 0)
        })

    return pd.DataFrame(records)

# %%
# 2.5.6 Generate and Save Synthetic Data
# -----------------------------------------------------------------------------
if USE_SYNTHETIC_DATA:
    print("=" * 70)
    print("GENERATING SYNTHETIC DATA (Development Mode)")
    print("=" * 70)
    print("\n⚠️  This is SYNTHETIC data for testing purposes only.")
    print("    Replace with real SEC filing data before final analysis.\n")

    # Generate all datasets
    synthetic_consolidated = generate_synthetic_consolidated()
    print(f"✓ Consolidated financials: {len(synthetic_consolidated)} records")

    synthetic_segments = generate_synthetic_segments(synthetic_consolidated)
    print(f"✓ Segment financials: {len(synthetic_segments)} records")

    synthetic_forward = generate_synthetic_forward_guidance()
    print(f"✓ Forward guidance: {len(synthetic_forward)} records")

    synthetic_obligations = generate_synthetic_obligations(synthetic_consolidated)
    print(f"✓ Purchase obligations: {len(synthetic_obligations)} records")

    # Save to CSV
    synthetic_consolidated.to_csv(DATA_PATH + 'consolidated_financials.csv', index=False)
    synthetic_segments.to_csv(DATA_PATH + 'segment_financials.csv', index=False)
    synthetic_forward.to_csv(DATA_PATH + 'forward_guidance.csv', index=False)
    synthetic_obligations.to_csv(DATA_PATH + 'purchase_obligations.csv', index=False)

    print(f"\n✓ Synthetic data saved to: {DATA_PATH}")
    print("=" * 70)

    # Preview consolidated data
    print("\nSample Consolidated Data (FY2024):")
    display(synthetic_consolidated[synthetic_consolidated['fiscal_year'] == 2024].sort_values('revenue', ascending=False))

# =============================================================================
# SECTION 3: DATA INGESTION
# =============================================================================

# %% [markdown]
"""
## Section 3: Data Ingestion

Load all financial data from structured CSV files.
"""

# %%
# 3.1 Data Schema Definitions
# -----------------------------------------------------------------------------
# Expected columns for each data file

CONSOLIDATED_SCHEMA = {
    'ticker': str,
    'fiscal_year': int,
    'revenue': float,
    'cost_of_revenue': float,
    'gross_profit': float,
    'rd_expense': float,
    'operating_income': float,
    'net_income': float,
    'total_assets': float,
    'total_debt': float,
    'cash_and_equivalents': float,
    'operating_cash_flow': float,
    'capex': float,
    'depreciation_amortization': float
}

SEGMENT_SCHEMA = {
    'ticker': str,
    'fiscal_year': int,
    'segment_name': str,
    'segment_revenue': float,
    'segment_operating_income': float
}

FORWARD_GUIDANCE_SCHEMA = {
    'ticker': str,
    'guidance_year': int,
    'capex_guidance_low': float,
    'capex_guidance_high': float,
    'capex_guidance_midpoint': float,
    'source': str,
    'source_date': str
}

OBLIGATIONS_SCHEMA = {
    'ticker': str,
    'fiscal_year': int,
    'purchase_obligations_total': float,
    'purchase_obligations_1yr': float,
    'operating_lease_obligations': float,
    'finance_lease_obligations': float
}

# %%
# 3.2 Data Loading Functions
# -----------------------------------------------------------------------------
def load_consolidated_data(filepath: str) -> pd.DataFrame:
    """
    Load consolidated financial data with validation.

    Args:
        filepath: Path to consolidated_financials.csv

    Returns:
        DataFrame with consolidated financial data, or None if file not found
    """
    try:
        df = pd.read_csv(filepath)

        # Validate required columns
        required_cols = list(CONSOLIDATED_SCHEMA.keys())
        missing_cols = [col for col in required_cols if col not in df.columns]

        if missing_cols:
            raise ValueError(f"Missing columns: {missing_cols}")

        # Sort for consistency
        df = df.sort_values(['ticker', 'fiscal_year']).reset_index(drop=True)

        print(f"✓ Loaded consolidated data: {len(df)} records")
        print(f"  Companies: {df['ticker'].nunique()}")
        print(f"  Years: {sorted(df['fiscal_year'].unique())}")

        return df

    except FileNotFoundError:
        print(f"✗ File not found: {filepath}")
        return None
    except Exception as e:
        print(f"✗ Error loading consolidated data: {e}")
        return None


def load_segment_data(filepath: str) -> pd.DataFrame:
    """
    Load segment-level financial data with validation.

    Args:
        filepath: Path to segment_financials.csv

    Returns:
        DataFrame with segment data, or None if file not found
    """
    try:
        df = pd.read_csv(filepath)

        # Validate required columns
        required_cols = list(SEGMENT_SCHEMA.keys())
        missing_cols = [col for col in required_cols if col not in df.columns]

        if missing_cols:
            raise ValueError(f"Missing columns: {missing_cols}")

        df = df.sort_values(['ticker', 'fiscal_year', 'segment_name']).reset_index(drop=True)

        print(f"✓ Loaded segment data: {len(df)} records")

        return df

    except FileNotFoundError:
        print(f"✗ File not found: {filepath}")
        return None
    except Exception as e:
        print(f"✗ Error loading segment data: {e}")
        return None


def load_forward_guidance(filepath: str) -> pd.DataFrame:
    """
    Load forward CapEx guidance data.

    Args:
        filepath: Path to forward_guidance.csv

    Returns:
        DataFrame with forward guidance, or None if file not found
    """
    try:
        df = pd.read_csv(filepath)
        df = df.sort_values(['ticker', 'guidance_year']).reset_index(drop=True)

        print(f"✓ Loaded forward guidance: {len(df)} records")

        return df

    except FileNotFoundError:
        print(f"✗ File not found: {filepath}")
        return None
    except Exception as e:
        print(f"✗ Error loading forward guidance: {e}")
        return None


def load_obligations_data(filepath: str) -> pd.DataFrame:
    """
    Load commitments and obligations data.

    Args:
        filepath: Path to purchase_obligations.csv

    Returns:
        DataFrame with obligations data, or None if file not found
    """
    try:
        df = pd.read_csv(filepath)
        df = df.sort_values(['ticker', 'fiscal_year']).reset_index(drop=True)

        print(f"✓ Loaded obligations data: {len(df)} records")

        return df

    except FileNotFoundError:
        print(f"✗ File not found: {filepath}")
        return None
    except Exception as e:
        print(f"✗ Error loading obligations data: {e}")
        return None

# %%
# 3.3 Execute Data Loading
# -----------------------------------------------------------------------------
print("=" * 70)
print("LOADING DATA FILES")
print("=" * 70)

df_consolidated = load_consolidated_data(DATA_PATH + 'consolidated_financials.csv')
df_segment = load_segment_data(DATA_PATH + 'segment_financials.csv')
df_forward = load_forward_guidance(DATA_PATH + 'forward_guidance.csv')
df_obligations = load_obligations_data(DATA_PATH + 'purchase_obligations.csv')

# Store in dictionary for easy access
DATA = {
    'consolidated': df_consolidated,
    'segment': df_segment,
    'forward_guidance': df_forward,
    'obligations': df_obligations
}

# Summary
print("\n" + "=" * 70)
print("DATA LOADING SUMMARY")
print("=" * 70)
for name, df in DATA.items():
    status = f"{len(df)} records" if df is not None else "NOT LOADED"
    print(f"  {name}: {status}")

# =============================================================================
# SECTION 4: DATA VALIDATION & QUALITY CHECKS
# =============================================================================

# %% [markdown]
"""
## Section 4: Data Validation & Quality Checks

Ensure data integrity before calculations; flag anomalies.
"""

# %%
# 4.1 Completeness Check
# -----------------------------------------------------------------------------
def check_data_completeness(df: pd.DataFrame, expected_companies: list,
                            expected_years: list) -> Dict:
    """
    Verify all expected company-year combinations are present.

    Args:
        df: DataFrame to check
        expected_companies: List of ticker symbols
        expected_years: List of fiscal years

    Returns:
        Dictionary with completeness status and missing combinations
    """
    results = {'complete': True, 'missing': []}

    for ticker in expected_companies:
        for year in expected_years:
            mask = (df['ticker'] == ticker) & (df['fiscal_year'] == year)
            if mask.sum() == 0:
                results['complete'] = False
                results['missing'].append((ticker, year))

    return results

# %%
# 4.2 Reasonableness Checks
# -----------------------------------------------------------------------------
def check_data_reasonableness(df: pd.DataFrame) -> pd.DataFrame:
    """
    Flag records that fail basic sanity checks.

    Args:
        df: DataFrame with consolidated financial data

    Returns:
        DataFrame with flagged issues
    """
    flags = []

    for idx, row in df.iterrows():
        issues = []

        # Revenue should be positive
        if row['revenue'] <= 0:
            issues.append('negative_revenue')

        # Gross profit should not exceed revenue
        if row['gross_profit'] > row['revenue'] * 1.01:  # Allow 1% tolerance
            issues.append('gross_profit_exceeds_revenue')

        # Operating income should not exceed gross profit
        if row['operating_income'] > row['gross_profit'] * 1.01:
            issues.append('operating_income_exceeds_gross_profit')

        # CapEx should be positive
        if row['capex'] < 0:
            issues.append('negative_capex')

        # Cash should be positive
        if row['cash_and_equivalents'] < 0:
            issues.append('negative_cash')

        # Assets should exceed debt (for these companies)
        if row['total_debt'] > row['total_assets']:
            issues.append('debt_exceeds_assets')

        if issues:
            flags.append({
                'ticker': row['ticker'],
                'fiscal_year': row['fiscal_year'],
                'issues': ', '.join(issues)
            })

    return pd.DataFrame(flags)

# %%
# 4.3 Year-over-Year Change Checks
# -----------------------------------------------------------------------------
def check_yoy_changes(df: pd.DataFrame, threshold: float = 1.0) -> pd.DataFrame:
    """
    Flag unusual year-over-year changes that may indicate data errors.

    Args:
        df: DataFrame with consolidated financial data
        threshold: Maximum expected YoY change (1.0 = 100%)

    Returns:
        DataFrame with flagged unusual changes
    """
    flags = []
    metrics_to_check = ['revenue', 'capex', 'operating_income', 'total_assets']

    for ticker in df['ticker'].unique():
        company_df = df[df['ticker'] == ticker].sort_values('fiscal_year')

        for metric in metrics_to_check:
            values = company_df[metric].values
            years = company_df['fiscal_year'].values

            for i in range(1, len(values)):
                if values[i-1] != 0:
                    yoy_change = (values[i] - values[i-1]) / abs(values[i-1])

                    # Special handling for NVIDIA's exceptional growth
                    adjusted_threshold = threshold * 2 if ticker == 'NVDA' else threshold

                    if abs(yoy_change) > adjusted_threshold:
                        flags.append({
                            'ticker': ticker,
                            'fiscal_year': years[i],
                            'metric': metric,
                            'yoy_change_pct': round(yoy_change * 100, 1),
                            'prior_value': round(values[i-1], 0),
                            'current_value': round(values[i], 0)
                        })

    return pd.DataFrame(flags)

# %%
# 4.4 Execute Validation
# -----------------------------------------------------------------------------
print("=" * 70)
print("DATA VALIDATION REPORT")
print("=" * 70)

if df_consolidated is not None:
    # Completeness check
    completeness = check_data_completeness(
        df_consolidated,
        list(COMPANIES.keys()),
        FISCAL_YEARS
    )

    if completeness['complete']:
        print("\n✓ Data completeness: PASS")
        print(f"  All {len(COMPANIES)} companies × {len(FISCAL_YEARS)} years present")
    else:
        print("\n✗ Data completeness: FAIL")
        print(f"  Missing combinations: {completeness['missing']}")

    # Reasonableness check
    reasonableness_flags = check_data_reasonableness(df_consolidated)
    if len(reasonableness_flags) == 0:
        print("\n✓ Reasonableness checks: PASS")
        print("  No data quality issues detected")
    else:
        print(f"\n✗ Reasonableness checks: {len(reasonableness_flags)} issues found")
        display(reasonableness_flags)

    # YoY change check
    yoy_flags = check_yoy_changes(df_consolidated, threshold=1.0)
    if len(yoy_flags) == 0:
        print("\n✓ YoY change checks: PASS")
        print("  No unusual year-over-year changes detected")
    else:
        print(f"\n⚠ YoY change checks: {len(yoy_flags)} unusual changes (review recommended)")
        display(yoy_flags)

else:
    print("\n✗ Cannot run validation: consolidated data not loaded")

print("\n" + "=" * 70)

# =============================================================================
# SECTION 5: METRIC CALCULATIONS
# =============================================================================

# %% [markdown]
"""
## Section 5: Metric Calculations

Calculate all metrics required for the four dimensions of the scorecard.
"""

# %%
# 5.0 Helper Functions
# -----------------------------------------------------------------------------
def safe_divide(numerator: float, denominator: float, default: float = 0.0) -> float:
    """
    Safe division handling zero denominators and NaN values.

    Args:
        numerator: Dividend
        denominator: Divisor
        default: Value to return if division is not possible

    Returns:
        Result of division or default value
    """
    if denominator == 0 or pd.isna(denominator) or pd.isna(numerator):
        return default
    return numerator / denominator


def calculate_cagr(start_value: float, end_value: float, periods: int) -> float:
    """
    Calculate Compound Annual Growth Rate.

    Args:
        start_value: Beginning value
        end_value: Ending value
        periods: Number of periods

    Returns:
        CAGR as decimal (0.10 = 10%)
    """
    if start_value <= 0 or end_value <= 0 or periods <= 0:
        return np.nan
    return (end_value / start_value) ** (1 / periods) - 1

# %%
# 5.1 Capital Deployment Scale Metrics
# -----------------------------------------------------------------------------
def calculate_capital_deployment_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate Dimension 1: Capital Deployment Scale

    Metrics:
    - CapEx Intensity: CapEx / Revenue
    - CapEx Growth Rate: YoY % change in CapEx
    - CapEx Coverage: Operating Cash Flow / CapEx
    - Total Innovation Investment: (R&D + CapEx) / Revenue

    Args:
        df: DataFrame with consolidated financial data

    Returns:
        DataFrame with capital deployment metrics
    """
    results = []

    for ticker in df['ticker'].unique():
        company_df = df[df['ticker'] == ticker].sort_values('fiscal_year')

        for idx, row in company_df.iterrows():
            year = row['fiscal_year']

            # CapEx Intensity
            capex_intensity = safe_divide(row['capex'], row['revenue'])

            # CapEx Growth Rate (need prior year)
            prior_year_data = company_df[company_df['fiscal_year'] == year - 1]
            if len(prior_year_data) > 0:
                prior_capex = prior_year_data['capex'].values[0]
                capex_growth = safe_divide(row['capex'] - prior_capex, prior_capex)
            else:
                capex_growth = np.nan

            # CapEx Coverage
            capex_coverage = safe_divide(row['operating_cash_flow'], row['capex'])

            # Total Innovation Investment
            innovation_investment = safe_divide(
                row['rd_expense'] + row['capex'],
                row['revenue']
            )

            results.append({
                'ticker': ticker,
                'fiscal_year': year,
                'capex_intensity': capex_intensity,
                'capex_growth_rate': capex_growth,
                'capex_coverage': capex_coverage,
                'innovation_investment': innovation_investment
            })

    return pd.DataFrame(results)

# %%
# 5.2 Conversion Efficiency Metrics
# -----------------------------------------------------------------------------
def calculate_conversion_efficiency_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate Dimension 2: Conversion Efficiency

    Metrics:
    - Revenue Yield on CapEx: Revenue Growth $ / Prior Year CapEx
    - Incremental Revenue per CapEx $: ΔRevenue / ΔCapEx (rolling)
    - Asset Turnover: Revenue / Total Assets
    - Asset Turnover Trend: Change in asset turnover over analysis period

    Args:
        df: DataFrame with consolidated financial data

    Returns:
        DataFrame with conversion efficiency metrics
    """
    results = []

    for ticker in df['ticker'].unique():
        company_df = df[df['ticker'] == ticker].sort_values('fiscal_year')

        for idx, row in company_df.iterrows():
            year = row['fiscal_year']

            # Prior year data
            prior_year_data = company_df[company_df['fiscal_year'] == year - 1]

            # Revenue Yield on CapEx
            if len(prior_year_data) > 0:
                prior_revenue = prior_year_data['revenue'].values[0]
                prior_capex = prior_year_data['capex'].values[0]
                revenue_growth = row['revenue'] - prior_revenue
                revenue_yield = safe_divide(revenue_growth, prior_capex)
            else:
                revenue_yield = np.nan

            # Incremental Revenue per CapEx (need 2 prior years for delta)
            two_years_prior = company_df[company_df['fiscal_year'] == year - 2]
            if len(prior_year_data) > 0 and len(two_years_prior) > 0:
                delta_revenue = row['revenue'] - two_years_prior['revenue'].values[0]
                delta_capex = row['capex'] + prior_year_data['capex'].values[0]
                incremental_efficiency = safe_divide(delta_revenue, delta_capex)
            else:
                incremental_efficiency = np.nan

            # Asset Turnover
            asset_turnover = safe_divide(row['revenue'], row['total_assets'])

            results.append({
                'ticker': ticker,
                'fiscal_year': year,
                'revenue_yield_on_capex': revenue_yield,
                'incremental_revenue_per_capex': incremental_efficiency,
                'asset_turnover': asset_turnover
            })

    # Calculate Asset Turnover Trend (change over analysis period)
    results_df = pd.DataFrame(results)

    for ticker in results_df['ticker'].unique():
        mask = results_df['ticker'] == ticker
        company_data = results_df[mask].sort_values('fiscal_year')

        if len(company_data) >= 2:
            first_at = company_data['asset_turnover'].iloc[0]
            last_at = company_data['asset_turnover'].iloc[-1]
            trend = last_at - first_at
        else:
            trend = np.nan

        results_df.loc[mask, 'asset_turnover_trend'] = trend

    return results_df

# %%
# 5.3 Return Quality Metrics
# -----------------------------------------------------------------------------
def calculate_return_quality_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate Dimension 3: Return Quality

    Metrics:
    - Gross Margin: Gross Profit / Revenue
    - Gross Margin Change: Current GM - Prior Year GM
    - Operating Leverage: Δ Operating Income / Δ Revenue
    - FCF Margin: (Operating Cash Flow - CapEx) / Revenue
    - FCF Margin Trend: Change over analysis period

    Args:
        df: DataFrame with consolidated financial data

    Returns:
        DataFrame with return quality metrics
    """
    results = []

    for ticker in df['ticker'].unique():
        company_df = df[df['ticker'] == ticker].sort_values('fiscal_year')

        for idx, row in company_df.iterrows():
            year = row['fiscal_year']

            # Gross Margin
            gross_margin = safe_divide(row['gross_profit'], row['revenue'])

            # Prior year for comparisons
            prior_year_data = company_df[company_df['fiscal_year'] == year - 1]

            # Gross Margin Change
            if len(prior_year_data) > 0:
                prior_gm = safe_divide(
                    prior_year_data['gross_profit'].values[0],
                    prior_year_data['revenue'].values[0]
                )
                gm_change = gross_margin - prior_gm
            else:
                gm_change = np.nan

            # Operating Leverage
            if len(prior_year_data) > 0:
                delta_oi = row['operating_income'] - prior_year_data['operating_income'].values[0]
                delta_rev = row['revenue'] - prior_year_data['revenue'].values[0]
                operating_leverage = safe_divide(delta_oi, delta_rev) if delta_rev != 0 else np.nan
            else:
                operating_leverage = np.nan

            # FCF Margin
            fcf = row['operating_cash_flow'] - row['capex']
            fcf_margin = safe_divide(fcf, row['revenue'])

            results.append({
                'ticker': ticker,
                'fiscal_year': year,
                'gross_margin': gross_margin,
                'gross_margin_change': gm_change,
                'operating_leverage': operating_leverage,
                'fcf_margin': fcf_margin
            })

    # Calculate FCF Margin Trend
    results_df = pd.DataFrame(results)

    for ticker in results_df['ticker'].unique():
        mask = results_df['ticker'] == ticker
        company_data = results_df[mask].sort_values('fiscal_year')

        if len(company_data) >= 2:
            first_fcf = company_data['fcf_margin'].iloc[0]
            last_fcf = company_data['fcf_margin'].iloc[-1]
            trend = last_fcf - first_fcf
        else:
            trend = np.nan

        results_df.loc[mask, 'fcf_margin_trend'] = trend

    return results_df

# %%
# 5.4 Sustainability & Risk Metrics
# -----------------------------------------------------------------------------
def calculate_sustainability_metrics(df: pd.DataFrame,
                                     df_obligations: pd.DataFrame = None) -> pd.DataFrame:
    """
    Calculate Dimension 4: Sustainability & Risk

    Metrics:
    - FCF After CapEx: Operating Cash Flow - CapEx
    - FCF After CapEx Margin: (OCF - CapEx) / Revenue
    - Net Debt to EBITDA: (Total Debt - Cash) / (Operating Income + D&A)
    - CapEx Commitment Risk: Purchase Obligations / Cash

    Args:
        df: DataFrame with consolidated financial data
        df_obligations: DataFrame with purchase obligations data (optional)

    Returns:
        DataFrame with sustainability metrics
    """
    results = []

    for ticker in df['ticker'].unique():
        company_df = df[df['ticker'] == ticker].sort_values('fiscal_year')

        for idx, row in company_df.iterrows():
            year = row['fiscal_year']

            # FCF After CapEx
            fcf_after_capex = row['operating_cash_flow'] - row['capex']
            fcf_after_capex_margin = safe_divide(fcf_after_capex, row['revenue'])

            # EBITDA proxy (Operating Income + D&A)
            ebitda = row['operating_income'] + row['depreciation_amortization']

            # Net Debt
            net_debt = row['total_debt'] - row['cash_and_equivalents']

            # Net Debt to EBITDA
            net_debt_to_ebitda = safe_divide(net_debt, ebitda) if ebitda > 0 else np.nan

            # CapEx Commitment Risk (from obligations data)
            commitment_risk = np.nan
            if df_obligations is not None:
                oblig_data = df_obligations[
                    (df_obligations['ticker'] == ticker) &
                    (df_obligations['fiscal_year'] == year)
                ]
                if len(oblig_data) > 0:
                    purchase_oblig = oblig_data['purchase_obligations_total'].values[0]
                    commitment_risk = safe_divide(purchase_oblig, row['cash_and_equivalents'])

            results.append({
                'ticker': ticker,
                'fiscal_year': year,
                'fcf_after_capex': fcf_after_capex,
                'fcf_after_capex_margin': fcf_after_capex_margin,
                'net_debt_to_ebitda': net_debt_to_ebitda,
                'capex_commitment_risk': commitment_risk
            })

    return pd.DataFrame(results)

# %%
# 5.5 Execute All Metric Calculations
# -----------------------------------------------------------------------------
print("=" * 70)
print("CALCULATING METRICS")
print("=" * 70)

if df_consolidated is not None:
    # Calculate each dimension's metrics
    metrics_deployment = calculate_capital_deployment_metrics(df_consolidated)
    print(f"\n✓ Capital Deployment metrics: {len(metrics_deployment)} records")

    metrics_conversion = calculate_conversion_efficiency_metrics(df_consolidated)
    print(f"✓ Conversion Efficiency metrics: {len(metrics_conversion)} records")

    metrics_return = calculate_return_quality_metrics(df_consolidated)
    print(f"✓ Return Quality metrics: {len(metrics_return)} records")

    metrics_sustainability = calculate_sustainability_metrics(
        df_consolidated,
        df_obligations
    )
    print(f"✓ Sustainability metrics: {len(metrics_sustainability)} records")

    # Merge all metrics into single DataFrame
    df_metrics = metrics_deployment.merge(
        metrics_conversion, on=['ticker', 'fiscal_year']
    ).merge(
        metrics_return, on=['ticker', 'fiscal_year']
    ).merge(
        metrics_sustainability, on=['ticker', 'fiscal_year']
    )

    print(f"\n✓ Combined metrics DataFrame: {df_metrics.shape[0]} rows × {df_metrics.shape[1]} columns")

    # Display metrics for current year
    print(f"\nMetrics Summary (FY{CURRENT_YEAR}):")
    display(df_metrics[df_metrics['fiscal_year'] == CURRENT_YEAR].round(3))

else:
    print("\n✗ Cannot calculate metrics: consolidated data not loaded")
    df_metrics = None

print("\n" + "=" * 70)

# =============================================================================
# SECTION 6: SCORING ENGINE
# =============================================================================

# %% [markdown]
"""
## Section 6: Scoring Engine

Convert raw metrics to 1-10 scores using percentile ranking within the peer group.
"""

# %%
# 6.1 Percentile Scoring Function
# -----------------------------------------------------------------------------
def percentile_score(values: pd.Series, higher_is_better: bool = True) -> pd.Series:
    """
    Convert values to 1-10 score based on percentile rank within peer group.

    Args:
        values: Series of metric values
        higher_is_better: If True, higher values get higher scores

    Returns:
        Series of scores from 1-10
    """
    # Handle NaN values
    valid_mask = ~values.isna()
    scores = pd.Series(index=values.index, dtype=float)

    if valid_mask.sum() == 0:
        return scores

    # Calculate percentile ranks (0-1)
    ranks = values[valid_mask].rank(pct=True)

    # Flip if lower is better
    if not higher_is_better:
        ranks = 1 - ranks

    # Scale to 1-10
    scores[valid_mask] = 1 + (ranks * 9)

    return scores

# %%
# 6.2 Metric Scoring Configuration
# -----------------------------------------------------------------------------
# Define direction for each metric (True = higher is better)
METRIC_DIRECTION = {
    # Capital Deployment
    'capex_intensity': True,  # Higher intensity = more commitment to AI
    'capex_growth_rate': True,  # Higher growth = accelerating investment
    'capex_coverage': True,  # Higher coverage = better self-funding
    'innovation_investment': True,  # Higher = more total investment

    # Conversion Efficiency
    'revenue_yield_on_capex': True,  # Higher yield = better conversion
    'incremental_revenue_per_capex': True,  # Higher = better marginal return
    'asset_turnover': True,  # Higher = more efficient asset use
    'asset_turnover_trend': True,  # Improving trend is better

    # Return Quality
    'gross_margin': True,  # Higher margin = better
    'gross_margin_change': True,  # Improving margin = better
    'operating_leverage': True,  # Higher leverage = better
    'fcf_margin': True,  # Higher FCF margin = better
    'fcf_margin_trend': True,  # Improving trend = better

    # Sustainability
    'fcf_after_capex': True,  # Higher = better
    'fcf_after_capex_margin': True,  # Higher = better
    'net_debt_to_ebitda': False,  # Lower leverage = better
    'capex_commitment_risk': False  # Lower commitment risk = better
}

# Metric to Dimension mapping
METRIC_TO_DIMENSION = {
    'capex_intensity': 'capital_deployment',
    'capex_growth_rate': 'capital_deployment',
    'capex_coverage': 'capital_deployment',
    'innovation_investment': 'capital_deployment',

    'revenue_yield_on_capex': 'conversion_efficiency',
    'incremental_revenue_per_capex': 'conversion_efficiency',
    'asset_turnover': 'conversion_efficiency',
    'asset_turnover_trend': 'conversion_efficiency',

    'gross_margin': 'return_quality',
    'gross_margin_change': 'return_quality',
    'operating_leverage': 'return_quality',
    'fcf_margin': 'return_quality',
    'fcf_margin_trend': 'return_quality',

    'fcf_after_capex': 'sustainability_risk',
    'fcf_after_capex_margin': 'sustainability_risk',
    'net_debt_to_ebitda': 'sustainability_risk',
    'capex_commitment_risk': 'sustainability_risk'
}

# %%
# 6.3 Score All Metrics
# -----------------------------------------------------------------------------
def calculate_metric_scores(df_metrics: pd.DataFrame,
                           year: int = None) -> pd.DataFrame:
    """
    Calculate scores for all metrics.

    Args:
        df_metrics: DataFrame with raw metric values
        year: If specified, score only for that year (cross-sectional)
              If None, score across all years (panel)

    Returns:
        DataFrame with '_score' columns added for each metric
    """
    if year is not None:
        df_to_score = df_metrics[df_metrics['fiscal_year'] == year].copy()
    else:
        df_to_score = df_metrics.copy()

    for metric, direction in METRIC_DIRECTION.items():
        if metric in df_to_score.columns:
            score_col = f"{metric}_score"
            df_to_score[score_col] = percentile_score(
                df_to_score[metric],
                higher_is_better=direction
            )

    return df_to_score

# %%
# 6.4 Calculate Dimension Scores
# -----------------------------------------------------------------------------
def calculate_dimension_scores(df_scored: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregate metric scores into dimension scores.

    Args:
        df_scored: DataFrame with individual metric scores

    Returns:
        DataFrame with dimension score columns added
    """
    df = df_scored.copy()

    for dimension in DIMENSION_WEIGHTS.keys():
        # Find all metrics for this dimension
        dimension_metrics = [
            m for m, d in METRIC_TO_DIMENSION.items()
            if d == dimension
        ]

        # Find corresponding score columns
        score_cols = [f"{m}_score" for m in dimension_metrics if f"{m}_score" in df.columns]

        if score_cols:
            # Calculate mean of available scores (handles NaN)
            df[f"{dimension}_score"] = df[score_cols].mean(axis=1, skipna=True)

    return df

# %%
# 6.5 Calculate Overall Score
# -----------------------------------------------------------------------------
def calculate_overall_score(df_dimension_scores: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate weighted overall efficiency score.

    Args:
        df_dimension_scores: DataFrame with dimension scores

    Returns:
        DataFrame with overall_score and classification columns
    """
    df = df_dimension_scores.copy()

    overall_score = pd.Series(0.0, index=df.index)

    for dimension, weight in DIMENSION_WEIGHTS.items():
        score_col = f"{dimension}_score"
        if score_col in df.columns:
            # Handle NaN by using 5.0 (neutral) as fallback
            dim_scores = df[score_col].fillna(5.0)
            overall_score += dim_scores * weight

    df['overall_score'] = overall_score

    # Add classification
    df['classification'] = df['overall_score'].apply(classify_score)

    return df


def classify_score(score: float) -> str:
    """
    Map score to classification tier.

    Args:
        score: Overall score (1-10)

    Returns:
        Classification string
    """
    if pd.isna(score):
        return 'Unclassified'

    for classification, (low, high) in CLASSIFICATION_THRESHOLDS.items():
        if low <= score <= high:
            return classification

    return 'Unclassified'

# %%
# 6.6 Execute Scoring
# -----------------------------------------------------------------------------
print("=" * 70)
print("CALCULATING SCORES")
print("=" * 70)

if df_metrics is not None:
    # Score for current year only (cross-sectional comparison)
    df_scored = calculate_metric_scores(df_metrics, year=CURRENT_YEAR)
    print(f"\n✓ Metric scores calculated for FY{CURRENT_YEAR}")

    # Calculate dimension scores
    df_scored = calculate_dimension_scores(df_scored)
    print("✓ Dimension scores calculated")

    # Calculate overall score
    df_final_scores = calculate_overall_score(df_scored)
    print("✓ Overall scores and classifications assigned")

    # Display summary
    summary_cols = [
        'ticker', 'fiscal_year',
        'capital_deployment_score', 'conversion_efficiency_score',
        'return_quality_score', 'sustainability_risk_score',
        'overall_score', 'classification'
    ]

    print(f"\n{'=' * 70}")
    print(f"SCORECARD RESULTS (FY{CURRENT_YEAR})")
    print(f"{'=' * 70}")

    display(
        df_final_scores[summary_cols]
        .sort_values('overall_score', ascending=False)
        .round(2)
    )

else:
    print("\n✗ Cannot calculate scores: metrics not available")
    df_final_scores = None

print("\n" + "=" * 70)

# =============================================================================
# SECTION 7: SEGMENT-LEVEL ANALYSIS
# =============================================================================

# %% [markdown]
"""
## Section 7: Segment-Level Analysis

Deep-dive into AI-primary segments for companies with segment reporting.
"""

# %%
# 7.1 Segment Metric Calculations
# -----------------------------------------------------------------------------
def calculate_segment_metrics(df_segment: pd.DataFrame,
                              df_consolidated: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate segment-level efficiency metrics.

    Args:
        df_segment: DataFrame with segment-level data
        df_consolidated: DataFrame with consolidated data

    Returns:
        DataFrame with segment metrics
    """
    results = []

    for _, row in df_segment.iterrows():
        ticker = row['ticker']
        year = row['fiscal_year']

        # Get consolidated data for this company-year
        consol_data = df_consolidated[
            (df_consolidated['ticker'] == ticker) &
            (df_consolidated['fiscal_year'] == year)
        ]

        if len(consol_data) == 0:
            continue

        total_revenue = consol_data['revenue'].values[0]

        # Segment Revenue Contribution
        revenue_contribution = safe_divide(row['segment_revenue'], total_revenue)

        # Segment Operating Margin
        segment_op_margin = safe_divide(
            row['segment_operating_income'],
            row['segment_revenue']
        )

        results.append({
            'ticker': ticker,
            'fiscal_year': year,
            'segment_name': row['segment_name'],
            'segment_revenue': row['segment_revenue'],
            'segment_operating_income': row['segment_operating_income'],
            'revenue_contribution': revenue_contribution,
            'segment_operating_margin': segment_op_margin
        })

    return pd.DataFrame(results)

# %%
# 7.2 AI-Primary Segment Comparison
# -----------------------------------------------------------------------------
def analyze_ai_primary_segments(df_segment_metrics: pd.DataFrame) -> pd.DataFrame:
    """
    Extract and compare AI-primary segments across companies.

    Args:
        df_segment_metrics: DataFrame with segment metrics

    Returns:
        DataFrame with AI-primary segment data only
    """
    ai_segments = []

    for ticker, config in SEGMENT_CONFIG.items():
        ai_segment_name = config['ai_primary']

        segment_data = df_segment_metrics[
            (df_segment_metrics['ticker'] == ticker) &
            (df_segment_metrics['segment_name'] == ai_segment_name)
        ]

        if len(segment_data) > 0:
            ai_segments.append(segment_data)

    if ai_segments:
        return pd.concat(ai_segments, ignore_index=True)
    return pd.DataFrame()

# %%
# 7.3 Segment Growth Analysis
# -----------------------------------------------------------------------------
def calculate_segment_growth(df_segment_metrics: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate YoY growth for segments.

    Args:
        df_segment_metrics: DataFrame with segment metrics

    Returns:
        DataFrame with segment growth metrics
    """
    results = []

    for ticker in df_segment_metrics['ticker'].unique():
        ticker_data = df_segment_metrics[df_segment_metrics['ticker'] == ticker]

        for segment in ticker_data['segment_name'].unique():
            segment_data = ticker_data[
                ticker_data['segment_name'] == segment
            ].sort_values('fiscal_year')

            for i in range(1, len(segment_data)):
                current = segment_data.iloc[i]
                prior = segment_data.iloc[i-1]

                # Revenue growth
                revenue_growth = safe_divide(
                    current['segment_revenue'] - prior['segment_revenue'],
                    prior['segment_revenue']
                )

                # Operating income growth (handle negative to positive transitions)
                if prior['segment_operating_income'] != 0:
                    oi_growth = safe_divide(
                        current['segment_operating_income'] - prior['segment_operating_income'],
                        abs(prior['segment_operating_income'])
                    )
                else:
                    oi_growth = np.nan

                # Margin change
                margin_change = current['segment_operating_margin'] - prior['segment_operating_margin']

                results.append({
                    'ticker': ticker,
                    'fiscal_year': current['fiscal_year'],
                    'segment_name': segment,
                    'revenue_growth': revenue_growth,
                    'operating_income_growth': oi_growth,
                    'margin_change': margin_change
                })

    return pd.DataFrame(results)

# %%
# 7.4 Execute Segment Analysis
# -----------------------------------------------------------------------------
print("=" * 70)
print("SEGMENT-LEVEL ANALYSIS")
print("=" * 70)

if df_segment is not None and df_consolidated is not None:
    # Calculate segment metrics
    df_segment_metrics = calculate_segment_metrics(df_segment, df_consolidated)
    print(f"\n✓ Segment metrics calculated: {len(df_segment_metrics)} records")

    # Extract AI-primary segments
    df_ai_segments = analyze_ai_primary_segments(df_segment_metrics)
    print(f"✓ AI-primary segments identified: {len(df_ai_segments)} records")

    # Calculate segment growth
    df_segment_growth = calculate_segment_growth(df_segment_metrics)
    print(f"✓ Segment growth calculated: {len(df_segment_growth)} records")

    # Display AI segment comparison for current year
    print(f"\n{'=' * 70}")
    print(f"AI-PRIMARY SEGMENT COMPARISON (FY{CURRENT_YEAR})")
    print(f"{'=' * 70}")

    ai_current = df_ai_segments[df_ai_segments['fiscal_year'] == CURRENT_YEAR].copy()
    ai_current['company'] = ai_current['ticker'].map(lambda x: COMPANIES[x]['name'])

    display(
        ai_current[['company', 'segment_name', 'segment_revenue',
                   'segment_operating_income', 'revenue_contribution',
                   'segment_operating_margin']]
        .sort_values('segment_operating_margin', ascending=False)
        .round(3)
    )

else:
    print("\n✗ Cannot perform segment analysis: data not available")
    df_segment_metrics = None
    df_ai_segments = None
    df_segment_growth = None

print("\n" + "=" * 70)

# =============================================================================
# SECTION 8: FORWARD GUIDANCE INTEGRATION
# =============================================================================

# %% [markdown]
"""
## Section 8: Forward Guidance Integration

Incorporate FY2025 CapEx guidance as a forward indicator.
"""

# %%
# 8.1 Forward CapEx Analysis
# -----------------------------------------------------------------------------
def analyze_forward_guidance(df_forward: pd.DataFrame,
                             df_consolidated: pd.DataFrame) -> pd.DataFrame:
    """
    Analyze forward CapEx guidance relative to historical patterns.

    Args:
        df_forward: DataFrame with forward guidance
        df_consolidated: DataFrame with historical data

    Returns:
        DataFrame with forward guidance analysis
    """
    results = []

    for ticker in df_forward['ticker'].unique():
        # Get latest guidance
        guidance_data = df_forward[df_forward['ticker'] == ticker]
        if len(guidance_data) == 0:
            continue
        guidance = guidance_data.iloc[-1]

        # Get historical data
        historical = df_consolidated[df_consolidated['ticker'] == ticker]
        if len(historical) == 0:
            continue

        # Get latest actual year
        latest_year = historical['fiscal_year'].max()
        latest_actual = historical[historical['fiscal_year'] == latest_year].iloc[0]

        # Calculate guidance vs. actual
        guidance_vs_actual = safe_divide(
            guidance['capex_guidance_midpoint'],
            latest_actual['capex']
        ) - 1

        # Calculate guidance as % of latest revenue
        guidance_intensity = safe_divide(
            guidance['capex_guidance_midpoint'],
            latest_actual['revenue']
        )

        # Historical average CapEx intensity
        historical_intensity = (historical['capex'] / historical['revenue']).mean()

        # Intensity change
        intensity_change = guidance_intensity - historical_intensity

        results.append({
            'ticker': ticker,
            'company': COMPANIES[ticker]['name'],
            'guidance_year': int(guidance['guidance_year']),
            'capex_guidance_midpoint': guidance['capex_guidance_midpoint'],
            'latest_actual_capex': latest_actual['capex'],
            'guidance_vs_actual_pct': guidance_vs_actual,
            'guidance_capex_intensity': guidance_intensity,
            'historical_avg_intensity': historical_intensity,
            'intensity_change': intensity_change,
            'source': guidance['source'],
            'source_date': guidance['source_date']
        })

    return pd.DataFrame(results)

# %%
# 8.2 Forward Indicator Scoring
# -----------------------------------------------------------------------------
def score_forward_guidance(df_forward_analysis: pd.DataFrame) -> pd.DataFrame:
    """
    Classify companies based on forward guidance trajectory.

    Args:
        df_forward_analysis: DataFrame with forward guidance analysis

    Returns:
        DataFrame with investment trajectory classification
    """
    df = df_forward_analysis.copy()

    def classify_trajectory(intensity_change):
        """Classify investment trajectory based on intensity change."""
        if pd.isna(intensity_change):
            return 'Unknown'
        if intensity_change > 0.02:  # >2% increase in intensity
            return 'Accelerating'
        elif intensity_change < -0.02:  # >2% decrease
            return 'Decelerating'
        else:
            return 'Stable'

    df['investment_trajectory'] = df['intensity_change'].apply(classify_trajectory)

    return df

# %%
# 8.3 Execute Forward Guidance Analysis
# -----------------------------------------------------------------------------
print("=" * 70)
print("FORWARD GUIDANCE ANALYSIS")
print("=" * 70)

if df_forward is not None and df_consolidated is not None:
    # Analyze forward guidance
    df_forward_analysis = analyze_forward_guidance(df_forward, df_consolidated)
    print(f"\n✓ Forward guidance analyzed: {len(df_forward_analysis)} companies")

    # Score and classify trajectories
    df_forward_scored = score_forward_guidance(df_forward_analysis)
    print("✓ Investment trajectories classified")

    # Display results
    print(f"\n{'=' * 70}")
    print(f"FY{FORWARD_YEAR} CAPEX GUIDANCE ANALYSIS")
    print(f"{'=' * 70}")

    display_cols = [
        'company', 'guidance_year', 'capex_guidance_midpoint',
        'latest_actual_capex', 'guidance_vs_actual_pct',
        'intensity_change', 'investment_trajectory'
    ]

    display(
        df_forward_scored[display_cols]
        .sort_values('guidance_vs_actual_pct', ascending=False)
        .round(3)
    )

else:
    print("\n✗ Cannot analyze forward guidance: data not available")
    df_forward_analysis = None
    df_forward_scored = None

print("\n" + "=" * 70)

# =============================================================================
# SECTION 9: VISUALIZATIONS
# =============================================================================

# %% [markdown]
"""
## Section 9: Visualizations

Create executive-ready charts for the scorecard.
"""

# %%
# 9.1 Scorecard Heatmap
# -----------------------------------------------------------------------------
def create_scorecard_heatmap(df_scores: pd.DataFrame, year: int) -> go.Figure:
    """
    Create heatmap showing dimension scores by company.

    Args:
        df_scores: DataFrame with final scores
        year: Fiscal year to display

    Returns:
        Plotly Figure object
    """
    df_year = df_scores[df_scores['fiscal_year'] == year].copy()
    df_year = df_year.sort_values('overall_score', ascending=True)

    # Prepare data for heatmap
    companies = [COMPANIES[t]['name'] for t in df_year['ticker']]
    dimensions = ['Capital\nDeployment', 'Conversion\nEfficiency',
                  'Return\nQuality', 'Sustainability\n& Risk', 'Overall']

    score_cols = ['capital_deployment_score', 'conversion_efficiency_score',
                  'return_quality_score', 'sustainability_risk_score', 'overall_score']

    z_data = df_year[score_cols].values

    # Create heatmap
    fig = go.Figure(data=go.Heatmap(
        z=z_data,
        x=dimensions,
        y=companies,
        colorscale='RdYlGn',
        zmin=1,
        zmax=10,
        text=np.round(z_data, 1),
        texttemplate='%{text}',
        textfont={'size': 12, 'color': 'black'},
        hovertemplate='%{y}<br>%{x}: %{z:.1f}<extra></extra>',
        colorbar=dict(title='Score', tickvals=[1, 4, 7, 10])
    ))

    fig.update_layout(
        title=dict(
            text=f'Capital Allocation Efficiency Scorecard (FY{year})',
            font=dict(size=18)
        ),
        xaxis_title='',
        yaxis_title='',
        template=CHART_TEMPLATE,
        height=450,
        width=800,
        margin=dict(l=120, r=50, t=80, b=50)
    )

    return fig

# %%
# 9.2 Dimension Comparison Radar Chart (CORRECTED)
# -----------------------------------------------------------------------------
def create_radar_chart(df_scores: pd.DataFrame, year: int,
                       companies_to_show: list = None) -> go.Figure:
    """
    Create radar chart comparing companies across dimensions.
    Uses small multiples layout for clarity.
    """
    from plotly.subplots import make_subplots

    df_year = df_scores[df_scores['fiscal_year'] == year].copy()
    df_year = df_year.sort_values('overall_score', ascending=False)

    if companies_to_show:
        df_year = df_year[df_year['ticker'].isin(companies_to_show)]

    categories = ['Conversion<br>Efficiency', 'Capital<br>Deployment',
                  'Sustainability<br>& Risk', 'Return<br>Quality']

    # Create 2x4 subplot grid
    fig = make_subplots(
        rows=2, cols=4,
        specs=[[{'type': 'polar'}] * 4, [{'type': 'polar'}] * 4],
        subplot_titles=[COMPANIES[t]['name'] for t in df_year['ticker'].tolist()] + [''],
        horizontal_spacing=0.08,
        vertical_spacing=0.12
    )

    positions = [(1,1), (1,2), (1,3), (1,4), (2,1), (2,2), (2,3)]

    for idx, (_, row) in enumerate(df_year.iterrows()):
        if idx >= 7:
            break

        ticker = row['ticker']
        color = COMPANIES[ticker]['color']
        r, c = positions[idx]

        values = [
            row['conversion_efficiency_score'],
            row['capital_deployment_score'],
            row['sustainability_risk_score'],
            row['return_quality_score']
        ]
        values.append(values[0])

        fig.add_trace(
            go.Scatterpolar(
                r=values,
                theta=categories + [categories[0]],
                name=COMPANIES[ticker]['name'],
                line=dict(color=color, width=2),
                fill='toself',
                fillcolor=color,
                opacity=0.4,
                showlegend=False
            ),
            row=r, col=c
        )

    # Update all polar axes
    for i in range(1, 8):
        fig.update_polars(
            radialaxis=dict(
                visible=True,
                range=[0, 10],
                tickvals=[2, 4, 6, 8, 10],
                tickfont=dict(size=8)
            ),
            angularaxis=dict(
                tickfont=dict(size=9)
            ),
            selector=dict(subplot=f'polar{i}' if i > 1 else 'polar')
        )

    fig.update_layout(
        title=dict(
            text=f'Dimension Comparison by Company (FY{year})<br><sup>Ranked by Overall Score: Highest (top-left) to Lowest (bottom-right)</sup>',
            font=dict(size=16),
            x=0.5
        ),
        template=CHART_TEMPLATE,
        height=700,
        width=1000,
        showlegend=False
    )

    return fig

# %%
# 9.3 CapEx Intensity Trend Chart
# -----------------------------------------------------------------------------
def create_capex_trend_chart(df_metrics: pd.DataFrame) -> go.Figure:
    """
    Create line chart showing CapEx intensity trends over time.

    Args:
        df_metrics: DataFrame with calculated metrics

    Returns:
        Plotly Figure object
    """
    fig = go.Figure()

    for ticker in COMPANY_ORDER:
        company_data = df_metrics[df_metrics['ticker'] == ticker].sort_values('fiscal_year')

        fig.add_trace(go.Scatter(
            x=company_data['fiscal_year'],
            y=company_data['capex_intensity'] * 100,  # Convert to percentage
            name=COMPANIES[ticker]['name'],
            line=dict(color=COMPANIES[ticker]['color'], width=2.5),
            mode='lines+markers',
            marker=dict(size=8)
        ))

    fig.update_layout(
        title=dict(
            text='CapEx Intensity Trend (% of Revenue)',
            font=dict(size=18)
        ),
        xaxis_title='Fiscal Year',
        yaxis_title='CapEx / Revenue (%)',
        template=CHART_TEMPLATE,
        height=450,
        width=850,
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.25,
            xanchor='center',
            x=0.5
        ),
        xaxis=dict(tickmode='array', tickvals=FISCAL_YEARS),
        yaxis=dict(ticksuffix='%')
    )

    return fig

# %%
# 9.4 Monetization Velocity Quadrant (CORRECTED)
# -----------------------------------------------------------------------------
def create_velocity_quadrant(df_metrics: pd.DataFrame, year: int) -> go.Figure:
    """
    Create quadrant chart: CapEx Intensity vs. Revenue Yield.
    Visualizes the Monetization Velocity concept with smart label positioning.
    """
    df_year = df_metrics[df_metrics['fiscal_year'] == year].copy()
    df_year = df_year.dropna(subset=['capex_intensity', 'revenue_yield_on_capex'])

    fig = go.Figure()

    # Custom label positions to avoid overlap
    label_adjustments = {
        'NVDA': (0, 3, 'top center'),
        'AAPL': (0, 1.5, 'top center'),
        'TSLA': (-1.5, 0, 'middle left'),
        'AMZN': (0, -1.5, 'bottom center'),
        'GOOGL': (1.5, 0, 'middle right'),
        'MSFT': (0, 1.5, 'top center'),
        'META': (0, -1.5, 'bottom center')
    }

    for _, row in df_year.iterrows():
        ticker = row['ticker']
        x_val = row['capex_intensity'] * 100
        y_val = row['revenue_yield_on_capex']

        x_offset, y_offset, text_pos = label_adjustments.get(ticker, (0, 0, 'top center'))

        # Add marker
        fig.add_trace(go.Scatter(
            x=[x_val],
            y=[y_val],
            mode='markers',
            name=COMPANIES[ticker]['name'],
            marker=dict(
                size=30,
                color=COMPANIES[ticker]['color'],
                line=dict(width=2, color='white')
            ),
            hovertemplate=f"<b>{COMPANIES[ticker]['name']}</b><br>" +
                          f"CapEx Intensity: {x_val:.1f}%<br>" +
                          f"Revenue Yield: {y_val:.1f}x<extra></extra>"
        ))

        # Add label with offset
        fig.add_annotation(
            x=x_val,
            y=y_val + y_offset,
            text=f"<b>{COMPANIES[ticker]['name']}</b>",
            showarrow=True if y_offset != 0 or x_offset != 0 else False,
            arrowhead=0,
            arrowsize=1,
            arrowwidth=1,
            arrowcolor='gray',
            ax=x_offset * 10,
            ay=-y_offset * 10 if y_offset > 0 else -y_offset * 10,
            font=dict(size=11, color='black'),
            align='center'
        )

    # Add quadrant lines
    median_intensity = df_year['capex_intensity'].median() * 100
    median_yield = df_year['revenue_yield_on_capex'].median()

    fig.add_hline(y=median_yield, line_dash='dash', line_color='gray', opacity=0.5)
    fig.add_vline(x=median_intensity, line_dash='dash', line_color='gray', opacity=0.5)

    # Add quadrant labels
    fig.add_annotation(x=0.02, y=0.98, xref='paper', yref='paper',
                       text='<b>Efficient</b><br><span style="font-size:10px">Low Investment, High Return</span>',
                       showarrow=False, font=dict(size=11, color='green'),
                       align='left', xanchor='left', yanchor='top')
    fig.add_annotation(x=0.98, y=0.98, xref='paper', yref='paper',
                       text='<b>Scaling</b><br><span style="font-size:10px">High Investment, High Return</span>',
                       showarrow=False, font=dict(size=11, color='blue'),
                       align='right', xanchor='right', yanchor='top')
    fig.add_annotation(x=0.02, y=0.02, xref='paper', yref='paper',
                       text='<b>Stagnant</b><br><span style="font-size:10px">Low Investment, Low Return</span>',
                       showarrow=False, font=dict(size=11, color='gray'),
                       align='left', xanchor='left', yanchor='bottom')
    fig.add_annotation(x=0.98, y=0.02, xref='paper', yref='paper',
                       text='<b>Investing</b><br><span style="font-size:10px">High Investment, Low Return</span>',
                       showarrow=False, font=dict(size=11, color='orange'),
                       align='right', xanchor='right', yanchor='bottom')

    fig.update_layout(
        title=dict(
            text=f'Monetization Velocity Quadrant (FY{year})',
            font=dict(size=18)
        ),
        xaxis_title='CapEx Intensity (% of Revenue)',
        yaxis_title='Revenue Yield on CapEx ($/$)',
        template=CHART_TEMPLATE,
        height=600,
        width=850,
        showlegend=False,
        xaxis=dict(
            ticksuffix='%',
            range=[0, 25],
            dtick=5
        ),
        yaxis=dict(
            range=[-2, 55],
            dtick=10
        )
    )

    return fig

# %%
# 9.5 AI Segment Profitability Comparison
# -----------------------------------------------------------------------------
def create_segment_margin_chart(df_ai_segments: pd.DataFrame, year: int) -> go.Figure:
    """
    Bar chart comparing AI-primary segment operating margins.

    Args:
        df_ai_segments: DataFrame with AI-primary segment data
        year: Fiscal year to display

    Returns:
        Plotly Figure object
    """
    df_year = df_ai_segments[df_ai_segments['fiscal_year'] == year].copy()
    df_year = df_year.sort_values('segment_operating_margin', ascending=True)

    colors = [COMPANIES[t]['color'] for t in df_year['ticker']]
    labels = [f"{COMPANIES[t]['name']}<br>({s})"
              for t, s in zip(df_year['ticker'], df_year['segment_name'])]

    fig = go.Figure(go.Bar(
        x=df_year['segment_operating_margin'] * 100,
        y=labels,
        orientation='h',
        marker_color=colors,
        text=[f"{m:.1f}%" for m in df_year['segment_operating_margin'] * 100],
        textposition='outside',
        textfont=dict(size=11)
    ))

    fig.update_layout(
        title=dict(
            text=f'AI-Primary Segment Operating Margins (FY{year})',
            font=dict(size=18)
        ),
        xaxis_title='Operating Margin (%)',
        yaxis_title='',
        template=CHART_TEMPLATE,
        height=450,
        width=750,
        xaxis=dict(ticksuffix='%', range=[-100, 100]),
        margin=dict(l=180)
    )

    # Add vertical line at 0%
    fig.add_vline(x=0, line_dash='solid', line_color='black', line_width=1)

    return fig

# %%
# 9.6 Forward Guidance Comparison
# -----------------------------------------------------------------------------
def create_forward_guidance_chart(df_forward_analysis: pd.DataFrame) -> go.Figure:
    """
    Bar chart comparing FY2025 CapEx guidance vs. FY2024 actual.

    Args:
        df_forward_analysis: DataFrame with forward guidance analysis

    Returns:
        Plotly Figure object
    """
    df = df_forward_analysis.sort_values('guidance_vs_actual_pct', ascending=True)

    colors = [COMPANIES[t]['color'] for t in df['ticker']]

    fig = go.Figure(go.Bar(
        x=df['guidance_vs_actual_pct'] * 100,
        y=df['company'],
        orientation='h',
        marker_color=colors,
        text=[f"{p:+.0f}%" for p in df['guidance_vs_actual_pct'] * 100],
        textposition='outside',
        textfont=dict(size=11)
    ))

    fig.update_layout(
        title=dict(
            text=f'FY{FORWARD_YEAR} CapEx Guidance vs. FY{CURRENT_YEAR} Actual',
            font=dict(size=18)
        ),
        xaxis_title='Change (%)',
        yaxis_title='',
        template=CHART_TEMPLATE,
        height=400,
        width=700,
        xaxis=dict(ticksuffix='%'),
        margin=dict(l=100)
    )

    # Add vertical line at 0%
    fig.add_vline(x=0, line_dash='solid', line_color='black', line_width=1)

    return fig

# %%
# 9.7 Generate All Visualizations
# -----------------------------------------------------------------------------
print("=" * 70)
print("GENERATING VISUALIZATIONS")
print("=" * 70)

# Store figures for export
figures = {}

if df_final_scores is not None:
    print("\n✓ Creating scorecard heatmap...")
    fig_heatmap = create_scorecard_heatmap(df_final_scores, CURRENT_YEAR)
    figures['scorecard_heatmap'] = fig_heatmap
    fig_heatmap.show()

    print("✓ Creating radar chart...")
    fig_radar = create_radar_chart(df_final_scores, CURRENT_YEAR)
    figures['radar_chart'] = fig_radar
    fig_radar.show()

if df_metrics is not None:
    print("✓ Creating CapEx trend chart...")
    fig_trend = create_capex_trend_chart(df_metrics)
    figures['capex_trend'] = fig_trend
    fig_trend.show()

    print("✓ Creating velocity quadrant...")
    fig_quadrant = create_velocity_quadrant(df_metrics, CURRENT_YEAR)
    figures['velocity_quadrant'] = fig_quadrant
    fig_quadrant.show()

if df_ai_segments is not None and len(df_ai_segments) > 0:
    print("✓ Creating segment margin chart...")
    fig_segments = create_segment_margin_chart(df_ai_segments, CURRENT_YEAR)
    figures['segment_margins'] = fig_segments
    fig_segments.show()

if df_forward_analysis is not None:
    print("✓ Creating forward guidance chart...")
    fig_forward = create_forward_guidance_chart(df_forward_analysis)
    figures['forward_guidance'] = fig_forward
    fig_forward.show()

print(f"\n✓ Generated {len(figures)} visualizations")
print("=" * 70)

# =============================================================================
# SECTION 10: EXECUTIVE SUMMARY GENERATOR
# =============================================================================

# %% [markdown]
"""
## Section 10: Executive Summary Generator

Auto-generate key findings text from model outputs.
"""

# %%
# 10.1 Generate Executive Summary
# -----------------------------------------------------------------------------
def generate_executive_summary(df_scores: pd.DataFrame,
                               df_metrics: pd.DataFrame,
                               df_forward: pd.DataFrame = None,
                               year: int = CURRENT_YEAR) -> str:
    """
    Generate executive summary text from model outputs.

    Args:
        df_scores: DataFrame with final scores
        df_metrics: DataFrame with calculated metrics
        df_forward: DataFrame with forward guidance (optional)
        year: Fiscal year for analysis

    Returns:
        Markdown-formatted summary string
    """
    df_year = df_scores[df_scores['fiscal_year'] == year].sort_values(
        'overall_score', ascending=False
    )

    # Top and bottom performers
    top_performer = df_year.iloc[0]
    bottom_performer = df_year.iloc[-1]

    # Classification counts
    classifications = df_year['classification'].value_counts().to_dict()

    # Score spread
    score_spread = top_performer['overall_score'] - bottom_performer['overall_score']

    # Build summary
    summary = f"""
# Executive Summary: Capital Allocation Efficiency Scorecard

**Analysis Period**: FY{year}
**Companies Analyzed**: {len(df_year)} (Magnificent Seven)
**Generated**: {datetime.now().strftime('%Y-%m-%d')}

---

## Key Finding

Among the Magnificent Seven, **{COMPANIES[top_performer['ticker']]['name']}** demonstrates
the highest capital allocation efficiency with an overall score of **{top_performer['overall_score']:.1f}/10**,
while **{COMPANIES[bottom_performer['ticker']]['name']}** scores lowest at **{bottom_performer['overall_score']:.1f}/10**.

The **{score_spread:.1f}-point spread** between highest and lowest scores indicates meaningful
differentiation in capital productivity across the peer group.

---

## Classification Distribution

"""

    for classification in ['Capital Efficient', 'Capital Intensive', 'Capital Dependent']:
        if classification in classifications:
            count = classifications[classification]
            companies_in_class = df_year[df_year['classification'] == classification]['ticker'].tolist()
            company_names = [COMPANIES[t]['name'] for t in companies_in_class]
            summary += f"**{classification}** ({count}): {', '.join(company_names)}\n\n"

    # Dimension leaders
    summary += """---

## Dimension Leaders

"""

    dimensions = {
        'conversion_efficiency_score': 'Conversion Efficiency',
        'return_quality_score': 'Return Quality',
        'sustainability_risk_score': 'Sustainability & Risk',
        'capital_deployment_score': 'Capital Deployment'
    }

    for col, name in dimensions.items():
        if col in df_year.columns:
            leader_idx = df_year[col].idxmax()
            leader_ticker = df_year.loc[leader_idx, 'ticker']
            leader_score = df_year.loc[leader_idx, col]
            summary += f"- **{name}**: {COMPANIES[leader_ticker]['name']} ({leader_score:.1f})\n"

    # Strategic implications
    summary += """
---

## Strategic Implications

1. **High Conversion Efficiency** companies are successfully translating CapEx into
   near-term revenue growth, validating their AI investment thesis.

2. **Companies in the "Capital Intensive" tier** face investor scrutiny on return timing.
   Monitor for margin improvement in coming quarters.

3. **Low Sustainability scores** indicate potential financial strain if AI investment
   continues at current pace without corresponding revenue acceleration.

---

## Methodology Note

This scorecard uses percentile ranking within the Magnificent Seven peer group.
Scores reflect relative positioning, not absolute thresholds. A company scoring
5.0 represents median performance among these seven technology leaders.

"""

    # Add forward guidance section if available
    if df_forward is not None and len(df_forward) > 0:
        accelerating = df_forward[df_forward['investment_trajectory'] == 'Accelerating']['company'].tolist()
        decelerating = df_forward[df_forward['investment_trajectory'] == 'Decelerating']['company'].tolist()

        summary += f"""---

## Forward Outlook (FY{FORWARD_YEAR})

Based on disclosed CapEx guidance:

"""
        if accelerating:
            summary += f"- **Accelerating Investment**: {', '.join(accelerating)}\n"
        if decelerating:
            summary += f"- **Decelerating Investment**: {', '.join(decelerating)}\n"

        summary += "\nMonitor Q1-Q2 earnings for guidance revisions as AI demand signals evolve.\n"

    return summary

# %%
# 10.2 Generate and Display Summary
# -----------------------------------------------------------------------------
print("=" * 70)
print("EXECUTIVE SUMMARY")
print("=" * 70)

if df_final_scores is not None and df_metrics is not None:
    executive_summary = generate_executive_summary(
        df_final_scores,
        df_metrics,
        df_forward_scored if 'df_forward_scored' in dir() else None,
        CURRENT_YEAR
    )

    from IPython.display import Markdown
    display(Markdown(executive_summary))
else:
    print("\n✗ Cannot generate summary: required data not available")
    executive_summary = None

!pip install -U kaleido -q

# =============================================================================
# SECTION 11: EXPORT FUNCTIONS
# =============================================================================

# %% [markdown]
"""
## Section 11: Export Functions

Save outputs for GitHub repository and external use.
"""

# %%
# 11.1 Export Data to CSV
# -----------------------------------------------------------------------------
def export_data_to_csv(output_path: str) -> List[str]:
    """
    Export all DataFrames to CSV files.

    Args:
        output_path: Directory path for output files

    Returns:
        List of exported file paths
    """
    exported_files = []

    exports = {
        'consolidated_financials': df_consolidated,
        'calculated_metrics': df_metrics if 'df_metrics' in dir() else None,
        'final_scores': df_final_scores if 'df_final_scores' in dir() else None,
        'segment_metrics': df_segment_metrics if 'df_segment_metrics' in dir() else None,
        'ai_primary_segments': df_ai_segments if 'df_ai_segments' in dir() else None,
        'forward_guidance_analysis': df_forward_scored if 'df_forward_scored' in dir() else None
    }

    for name, df in exports.items():
        if df is not None and len(df) > 0:
            filepath = f"{output_path}{name}.csv"
            df.to_csv(filepath, index=False)
            exported_files.append(filepath)
            print(f"✓ Exported: {name}.csv")

    return exported_files

# %%
# 11.2 Export Visualizations
# -----------------------------------------------------------------------------
def export_visualizations(output_path: str, figures_dict: dict) -> List[str]:
    """
    Export all figures as PNG and HTML files.

    Args:
        output_path: Directory path for output files
        figures_dict: Dictionary of figure name -> Plotly Figure

    Returns:
        List of exported file paths
    """
    exported_files = []

    for name, fig in figures_dict.items():
        if fig is not None:
            try:
                # Export as PNG
                png_path = f"{output_path}{name}.png"
                fig.write_image(png_path, scale=2)
                exported_files.append(png_path)

                # Export as interactive HTML
                html_path = f"{output_path}{name}.html"
                fig.write_html(html_path)
                exported_files.append(html_path)

                print(f"✓ Exported: {name}.png and {name}.html")
            except Exception as e:
                print(f"⚠ Could not export {name}: {e}")

    return exported_files

# %%
# 11.3 Export Executive Summary
# -----------------------------------------------------------------------------
def export_summary(output_path: str, summary_text: str) -> str:
    """
    Export executive summary as Markdown file.

    Args:
        output_path: Directory path for output file
        summary_text: Markdown-formatted summary

    Returns:
        Path to exported file
    """
    filepath = f"{output_path}executive_summary.md"

    with open(filepath, 'w') as f:
        f.write(summary_text)

    print(f"✓ Exported: executive_summary.md")
    return filepath

# %%
# 11.4 Execute All Exports
# -----------------------------------------------------------------------------
print("=" * 70)
print("EXPORTING OUTPUTS")
print("=" * 70)

# Ensure output directory exists
os.makedirs(OUTPUT_PATH, exist_ok=True)

# Export data
print("\nExporting data files...")
export_data_to_csv(OUTPUT_PATH)

# Export visualizations
if 'figures' in dir() and len(figures) > 0:
    print("\nExporting visualizations...")
    export_visualizations(OUTPUT_PATH, figures)

# Export executive summary
if 'executive_summary' in dir() and executive_summary is not None:
    print("\nExporting executive summary...")
    export_summary(OUTPUT_PATH, executive_summary)

print(f"\n{'=' * 70}")
print(f"✓ All exports complete")
print(f"  Output location: {OUTPUT_PATH}")
print(f"{'=' * 70}")

# =============================================================================
# SECTION 12: MODEL REFRESH UTILITIES
# =============================================================================

# %% [markdown]
"""
## Section 12: Model Refresh Utilities

Streamline updates when new 10-K filings become available.
"""

# %%
# 12.1 Data Refresh Checklist Generator
# -----------------------------------------------------------------------------
def generate_refresh_checklist(target_year: int) -> str:
    """
    Generate checklist for updating model with new fiscal year data.

    Args:
        target_year: The new fiscal year to add

    Returns:
        Markdown-formatted checklist
    """
    checklist = f"""
# Model Refresh Checklist: FY{target_year}

## Expected Filing Dates

| Company | Fiscal Year End | Expected 10-K Filing |
|---------|-----------------|---------------------|
"""

    filing_dates = {
        'AAPL': 'Late October',
        'MSFT': 'Late July',
        'GOOGL': 'Early February',
        'AMZN': 'Early February',
        'META': 'Early February',
        'NVDA': 'Late February',
        'TSLA': 'Late January'
    }

    for ticker, config in COMPANIES.items():
        checklist += f"| {config['name']} | {config['fy_end']} | {filing_dates[ticker]} |\n"

    checklist += f"""
---

## Data Collection Tasks

"""

    for ticker, config in COMPANIES.items():
        checklist += f"""
### {config['name']} ({ticker})

- [ ] Download 10-K filing from SEC EDGAR
- [ ] Extract consolidated financial data
- [ ] Extract segment data (if applicable)
- [ ] Update forward guidance from latest earnings call
- [ ] Verify purchase obligations from Notes
- [ ] Cross-check against earnings release figures

"""

    checklist += f"""
---

## Configuration Updates

After collecting all data:

1. [ ] Update `FISCAL_YEARS` list: `[{target_year-2}, {target_year-1}, {target_year}]`
2. [ ] Update `CURRENT_YEAR`: `{target_year}`
3. [ ] Update `FORWARD_YEAR`: `{target_year + 1}`
4. [ ] Set `USE_SYNTHETIC_DATA = False`

---

## Validation Steps

- [ ] Run data completeness check (Section 4)
- [ ] Review reasonableness check results
- [ ] Investigate any YoY change flags
- [ ] Verify segment data reconciles to consolidated

---

## Output Updates

- [ ] Regenerate all visualizations
- [ ] Update executive summary
- [ ] Export new CSV files
- [ ] Commit changes to GitHub repository
- [ ] Update README with new findings

"""

    return checklist

# %%
# 12.2 Quick Refresh Function
# -----------------------------------------------------------------------------
def refresh_model_config(new_year: int):
    """
    Quick configuration refresh for new fiscal year.

    Note: This updates global variables. Re-run Sections 3-11 after calling.

    Args:
        new_year: The new current fiscal year
    """
    global CURRENT_YEAR, FORWARD_YEAR, FISCAL_YEARS

    # Update configuration
    FISCAL_YEARS = [new_year - 2, new_year - 1, new_year]
    CURRENT_YEAR = new_year
    FORWARD_YEAR = new_year + 1

    print(f"{'=' * 70}")
    print("CONFIGURATION UPDATED")
    print(f"{'=' * 70}")
    print(f"\n  Analysis period: {FISCAL_YEARS}")
    print(f"  Current year: FY{CURRENT_YEAR}")
    print(f"  Forward year: FY{FORWARD_YEAR}")
    print(f"\n⚠️  Re-run Sections 3-11 to complete the refresh.")
    print(f"{'=' * 70}")

# %%
# 12.3 Display Refresh Checklist
# -----------------------------------------------------------------------------
print("=" * 70)
print("MODEL REFRESH UTILITIES")
print("=" * 70)

# Generate checklist for next refresh cycle
next_year = CURRENT_YEAR + 1
checklist = generate_refresh_checklist(next_year)

print(f"\nRefresh checklist generated for FY{next_year}")
print("Uncomment the line below to display the full checklist:")
print("# display(Markdown(checklist))")

# To refresh for a new year, uncomment and run:
# refresh_model_config(2025)

print("\n" + "=" * 70)
print("NOTEBOOK EXECUTION COMPLETE")
print("=" * 70)

if USE_SYNTHETIC_DATA:
    print("\n⚠️  REMINDER: This analysis used SYNTHETIC data.")
    print("   Set USE_SYNTHETIC_DATA = False and provide real SEC data")
    print("   before using results for decision-making.")